In [7]:
import sys
import os

# Add the project root directory to sys.path
project_root = os.path.abspath("..")  # Adjust this path if your notebook is in a different location
if project_root not in sys.path:
    sys.path.append(project_root)

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib

import src.data_preprocessing as prp
import src.feature_engineering as ftr

# Reload the module to reflect the changes
importlib.reload(prp)
importlib.reload(ftr)

<module 'src.feature_engineering' from 'd:\\work\\projects\\U of T\\CSC311\\final_project\\Credit-Risk-Prediction\\src\\feature_engineering.py'>

In [9]:
# Load the dataset

accepted_data, rejected_data = prp.load_data(processed=True)

Since we only care about specific columns, we subset the data on these columns.<br>

Columns:
- 

In [ ]:
accepted_data = accepted_data[""]

In [10]:
# Combine the data for training the isolation forest
combined_data = pd.concat([accepted_data, rejected_data], axis=0)

In [23]:
rejected_data.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [22]:
# Convert categorical variables to numerical
# sparse = True is used to save memory
combined_data_numerical = pd.get_dummies(combined_data, drop_first=True,  sparse=True)

MemoryError: Unable to allocate 452. MiB for an array with shape (2, 29628741) and data type object

In [ ]:
# Sample a smaller subset of the data for training the isolation forest
sampled_data = combined_data_numerical.sample(frac=0.01, random_state=42)

# Train the isolation forest
isolation_forest = ftr.create_isolation_forest(sampled_data)

ValueError: could not convert string to float: ' 60 months'